In [1]:
import sqlite3
import pandas as pd
import json

In [2]:
DATABASE = "./recruit.db"

In [3]:
conn = sqlite3.connect(DATABASE)

In [4]:
education = pd.read_sql('SELECT * FROM education', conn)
race = pd.read_sql('SELECT * FROM race', conn)
insurance_segment = pd.read_sql('select * FROM insurance_segment', conn)
customer = pd.read_sql('select * FROM customer', conn)

In [9]:
type(customer['income'][0])

numpy.int64

In [ ]:
test.groupby('economic_stability').sum()['cnt'].max()

In [ ]:
customerjson = json.loads(customer.to_json(orient='split'))

In [ ]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]


def query_db(query):

    result_dict = {}

    try:
        connection = sqlite3.connect(DATABASE)

        cursor = connection.cursor()
        cursor.execute(query)
        result_dict = dictfetchall(cursor)

    except sqlite3.OperationalError as e:
        print("Db operation error", e)
        result_dict["error"] = str(e)
    except:
        e = sys.exc_info()[0]
        print("An error occurred with the database", e)
        result_dict["error"] = str(e)
    else:
        cursor.close()
        connection.close()

    return result_dict

In [ ]:
connection = sqlite3.connect(DATABASE)
cursor = connection.cursor()
query = 'SELECT customer.economic_stability, insurance_segment.value, count(customer.economic_stability) as cnt \
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id \
            GROUP BY customer.economic_stability, insurance_segment.value'
cursor.execute(query)


In [ ]:
insurance_segment

In [ ]:
customer.head()

In [ ]:
data = dict()

for key in insurance_segment['value']:
    data[key] = []

query = 'SELECT customer.income, insurance_segment.value\
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id'


connection = sqlite3.connect(DATABASE)

cursor = connection.cursor()
cursor.execute(query)

for row in cursor.fetchall():
    data[row[1]].append(row[0])

In [22]:
i = 'Solid Suburbans'
value = test.loc[test['value'] == i, 'income']
p75 = np.percentile(value, 75)
p25 = np.percentile(value, 25)
iqr = p75 - p25
max_num = p75 + 1.5*iqr if p75 + 1.5*iqr < max(value) else max(value)
min_num = p25 - 1.5*iqr if p25 - 1.5*iqr > min(value) else min(value)

In [24]:
max_num, p75

(226.0, 124.0)

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []

for i in test['value'].unique():
    data.append(
        go.Box(
            y = test.loc[test['value'] == i, 'income']
        ),
        go.Scatter(
            
        )
    )
    
py.iplot(data, filename = 'mmtest')

In [19]:
np.percentile([1,2,3,4,5], 50)

3.0

In [12]:
query = 'SELECT customer.income, insurance_segment.value\
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id'

test = pd.read_sql(query, conn)

In [28]:
a_dict = test.head().to_dict(orient='records')
b_dict = test.head().to_dict(orient='records')

c = [dict(a = a, b = b) for a, b in (a_dict, b_dict)]

ValueError: too many values to unpack (expected 2)

In [33]:
for a, b in zip(a_dict, b_dict):
    print(b)

{'income': 25, 'value': "Price Isn't Primary"}
{'income': 357, 'value': 'Solid Suburbans'}
{'income': 25, 'value': "Price Isn't Primary"}
{'income': 51, 'value': 'Solid Suburbans'}
{'income': 51, 'value': 'Solid Suburbans'}
